<a href="https://colab.research.google.com/github/urazovandriy-dotcom/laboratony4mp/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# 1. ВИЗНАЧЕННЯ ФУНКЦІЇ ТА НАЛАШТУВАННЯ
# Нелінійне алгебраїчне рівняння: f(x) = x^4 - 2x^3 + 2x^2 - 6x + 1 = 0

def f(x):
    """Основна функція."""
    return x**4 - 2*x**3 + 2*x**2 - 6*x + 1

def f_double_prime(x):
    """Друга похідна для аналізу збіжності методу хорд: f''(x) = 12x^2 - 12x + 4."""
    return 12*x**2 - 12*x + 4

# Точність
epsilon = 0.0001
print(f"Задана точність: epsilon = {epsilon}")
print("-" * 50)

Задана точність: epsilon = 0.0001
--------------------------------------------------


In [ ]:
# 2. ВІДОКРЕМЛЕННЯ КОРЕНІВ
def separate_roots(f, start, end, step):
    """Шукає інтервали [a, b], де f(a) * f(b) < 0."""
    intervals = []
    x = start
    while x < end:
        a = x
        b = x + step
        # Перевірка зміни знаку на кінцях інтервалу
        if f(a) * f(b) < 0:
            intervals.append((a, b))
        x += step
    return intervals

# Шукаємо корені на інтервалі [-2, 4] з кроком 0.1
start_range = -2.0
end_range = 4.0
step = 0.1
root_intervals = separate_roots(f, start_range, end_range, step)

print(f"Знайдені інтервали відокремлення коренів:")
if not root_intervals:
    print("Корені не знайдено на заданому інтервалі. Можливо, вони комплексні або потрібно збільшити інтервал.")
else:
    for i, (a, b) in enumerate(root_intervals):
        print(f"  Корінь {i+1}: [{a:.4f}, {b:.4f}] | f({a:.1f})={f(a):.4f}, f({b:.1f})={f(b):.4f}")
print("-" * 50)

Знайдені інтервали відокремлення коренів:
  Корінь 1: [0.1000, 0.2000] | f(0.1)=0.4181, f(0.2)=-0.1344
  Корінь 2: [2.2000, 2.3000] | f(2.2)=-0.3904, f(2.3)=1.4301
--------------------------------------------------


In [ ]:
# 3. УТОЧНЕННЯ КОРЕНІВ МЕТОДОМ ПОЛОВИННОГО ДІЛЕННЯ

def bisection_method(f, a, b, epsilon):
    """Уточнення кореня методом половинного ділення."""
    if f(a) * f(b) >= 0:
        return "Помилка: Немає зміни знаку.", 0

    iter_count = 0
    # Умова зупинки: |b - a| <= epsilon
    while (b - a) > epsilon:
        c = (a + b) / 2
        f_c = f(c)

        if abs(f_c) < 1e-15: # Точний нуль
            a, b = c, c
            break

        if f(a) * f_c < 0:
            b = c
        else:
            a = c
        iter_count += 1

    root = (a + b) / 2
    return root, iter_count

print("Метод Половинного Ділення:")
bisection_results = []
for i, (a, b) in enumerate(root_intervals):
    root, iters = bisection_method(f, a, b, epsilon)
    bisection_results.append(root)
    print(f" Корінь {i+1}: x ≈ {root:.{len(str(int(1/epsilon)))+1}f} (за {iters} ітерацій)")
print("-" * 50)

Метод Половинного Ділення:
 Корінь 1: x ≈ 0.175244 (за 10 ітерацій)
 Корінь 2: x ≈ 2.223291 (за 10 ітерацій)
--------------------------------------------------


In [ ]:
# 4. УТОЧНЕННЯ КОРЕНІВ МЕТОДОМ ХОРД

def chord_method(f, f_double_prime, a, b, epsilon):
    """Уточнення кореня методом хорд із фіксацією кінця."""
    if f(a) * f(b) >= 0:
        return "Помилка: Немає зміни знаку.", 0

    iter_count = 0

    # Визначення фіксованого кінця
    # Фіксуємо той кінець, де знак f(x) збігається зі знаком f''(x)

    # Оцінюємо знак другої похідної на інтервалі
    f_dd_mid = f_double_prime((a + b) / 2)

    a_k, b_k = a, b
    fixed_point = None

    if f(a) * f_dd_mid > 0:
        fixed_point = a_k # Фіксуємо a
        x_k = b_k         # Початкове наближення - b
    elif f(b) * f_dd_mid > 0:
        fixed_point = b_k # Фіксуємо b
        x_k = a_k         # Початкове наближення - a
    else:
        # У разі, якщо f''(x) змінює знак, метод може бути нестабільним.
        return "Помилка: Невизначений фіксований кінець.", 0

    f_fixed = f(fixed_point)

    while True:
        # Формула методу хорд
        x_next = x_k - f(x_k) * (fixed_point - x_k) / (f_fixed - f(x_k))

        # Умова зупинки: |x_next - x_k| <= epsilon
        if abs(x_next - x_k) <= epsilon or abs(f(x_next)) < epsilon:
            break

        x_k = x_next
        iter_count += 1

        if iter_count > 100: # Обмеження для уникнення нескінченного циклу
             return "Помилка: Перевищено ліміт ітерацій.", iter_count

    return x_next, iter_count

print("Метод Хорд:")
for i, (a, b) in enumerate(root_intervals):
    root, iters = chord_method(f, f_double_prime, a, b, epsilon)
    print(f" Корінь {i+1}: x ≈ {root:.{len(str(int(1/epsilon)))+1}f} (за {iters} ітерацій)")
print("-" * 50)

Метод Хорд:
 Корінь 1: x ≈ 0.175276 (за 1 ітерацій)
 Корінь 2: x ≈ 2.223277 (за 3 ітерацій)
--------------------------------------------------


In [ ]:
# 5. РЕЗУЛЬТАТИ
print("Остаточні результати (Точність 0.0001)")
if len(root_intervals) >= 1:
    print(f"Перший корінь: x1 ≈ {bisection_results[0]:.{len(str(int(1/epsilon)))+1}f}")
if len(root_intervals) >= 2:
    print(f"Другий корінь: x2 ≈ {bisection_results[1]:.{len(str(int(1/epsilon)))+1}f}")

Остаточні результати (Точність 0.0001)
Перший корінь: x1 ≈ 0.175244
Другий корінь: x2 ≈ 2.223291
